# Watch Me Code 3: Mapping with Folium

Folium is a Python wrapper library for the OpenStreetMaps api. It allows you to place data on a map in a variety of ways.


In [ ]:
! pip install folium

In [ ]:
import folium
import pandas as pd
import random 

In [ ]:
# we need to center the map in the middle of the US. I googled for the location.
CENTER_US = (39.8333333,-98.585522)
london = (51.5074, -0.1278)
map = folium.Map(location=CENTER_US, zoom_start=4)
map

## Map Pins

In [ ]:
# read in a data file of IP address to locations.
data = pd.read_csv('https://raw.githubusercontent.com/mafudge/datasets/master/clickstream/ip_lookup.csv')
data.sample(5)

In [ ]:
# Let's place each location on the map
for row in data.to_records():
    pos = (row['ApproxLat'],row['ApproxLng'])
    marker = folium.Marker(location=pos, 
                    popup="%s, %s" % (row['City'],row['State'])
                          )
    map.add_child(marker)
map

In [ ]:
# Same thing with a different icon and colors. Icons come from http://fontawesome.io/icons/ but its an older version.
colors = ['red', 'blue', 'green', 'purple', 'orange', 'darkred',
            'lightred', 'beige', 'darkblue', 'darkgreen', 'cadetblue',
            'darkpurple', 'pink', 'lightblue', 'lightgreen',
            'gray', 'black', 'lightgray']
for row in data.to_records():
    pos = (row['ApproxLat'],row['ApproxLng'])
    marker = folium.Marker(location=pos, 
                    popup="%s, %s" % (row['City'],row['State']),
                    icon = folium.Icon(color = random.choice(colors), icon='user')
                          )
    map.add_child(marker)
map

In [ ]:
# There are other map tiles available. See https://folium.readthedocs.io/en/latest/quickstart.html
# Instead of Markers we use circles colors are HTML color codes http://htmlcolorcodes.com/
# https://deparkes.co.uk/2016/06/10/folium-map-tiles/
CENTER_US = (39.8333333,-98.585522)
map2 = folium.Map(location=CENTER_US, zoom_start=4, tiles='stamenterrain')
for row in data.to_records():
    map2.add_child(folium.CircleMarker(location=(row['ApproxLat'],row['ApproxLng']), 
                    popup=row['City'], radius=10, color='#0000FF',  fill_color='#FF3333'))
    
map2

# Choropleths

Choropleths are cartographic overlays based on boundries defined in a geo JSON file.


In [ ]:
# State level geo-json overlay choropleth
CENTER_US = (39.8333333,-98.585522)
state_geojson = 'WMC3-us-states.json'
map3 = folium.Map(location=CENTER_US, zoom_start=4, tiles='stamentoner')
map3.choropleth(geo_data=state_geojson)
map3


In [ ]:
states = pd.read_csv('https://raw.githubusercontent.com/jasonong/List-of-US-States/master/states.csv')
state_counts = pd.DataFrame( {'Counts' : data['State']. value_counts() } ).sort_index()
state_counts['StateCode'] = state_counts.index
state_data = states.merge(state_counts, how="left", left_on='Abbreviation', right_on='StateCode')
state_data = state_data[['Abbreviation','Counts']]
state_data = state_data.fillna(0)
state_data


In [ ]:
CENTER_US = (39.8333333,-98.585522)
state_geojson = 'WMC3-us-states.json'
map3 = folium.Map(location=CENTER_US, zoom_start=4, tiles='stamentoner')
map3.choropleth(geo_data=state_geojson,data=state_data, columns=['Abbreviation','Counts'], 
                key_on ='feature.id', fill_color='BuGn', legend_name='Website Visitors')
map3

In [ ]:
# Here's a more straigtforward example with unemployment data:
unemployment = pd.read_csv('https://raw.githubusercontent.com/wrobstory/vincent/master/examples/data/US_Unemployment_Oct2012.csv')
state_geojson = 'WMC3-us-states.json'
map4 = folium.Map(location=CENTER_US, zoom_start=4, tiles=' Open Street Map')
map4.choropleth(geo_data=state_geojson,data=unemployment, 
                columns=['State','Unemployment'], key_on ='feature.id', fill_color='YlOrRd', legend_name='2012 US Unemployment')
map4
